## Neural Networks

## QUE : PREDICT THE BURNED AREA OF FOREST FIRES WITH NEURAL NETWORKS


### Import Necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier


### Import Dataset

In [ ]:
fire_data= pd.read_csv('/content/forestfires.csv')
fire_data

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,small


In [ ]:
fire_data.isna().sum()

month            0
day              0
FFMC             0
DMC              0
DC               0
ISI              0
temp             0
RH               0
wind             0
rain             0
area             0
dayfri           0
daymon           0
daysat           0
daysun           0
daythu           0
daytue           0
daywed           0
monthapr         0
monthaug         0
monthdec         0
monthfeb         0
monthjan         0
monthjul         0
monthjun         0
monthmar         0
monthmay         0
monthnov         0
monthoct         0
monthsep         0
size_category    0
dtype: int64

In [ ]:
fire_data.dtypes

month             object
day               object
FFMC             float64
DMC              float64
DC               float64
ISI              float64
temp             float64
RH                 int64
wind             float64
rain             float64
area             float64
dayfri             int64
daymon             int64
daysat             int64
daysun             int64
daythu             int64
daytue             int64
daywed             int64
monthapr           int64
monthaug           int64
monthdec           int64
monthfeb           int64
monthjan           int64
monthjul           int64
monthjun           int64
monthmar           int64
monthmay           int64
monthnov           int64
monthoct           int64
monthsep           int64
size_category     object
dtype: object

In [ ]:
fire_data_1=fire_data.drop(['dayfri', 'daymon', 'daysat', 'daysun', 'daythu','daytue', 'daywed', 'monthapr', 'monthaug', 'monthdec', 
                  'monthfeb','monthjan', 'monthjul', 'monthjun', 'monthmar', 'monthmay', 'monthnov','monthoct','monthsep'], 
                 axis = 1)
fire_data_1


,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,small
...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,small


In [ ]:
fire_data_1['month']=fire_data_1['month'].astype('category')
fire_data_1['day']=fire_data_1['day'].astype('category')
fire_data_1['size_category']=fire_data_1['size_category'].astype('category')

In [ ]:
fire_data_1['month']=fire_data_1['month'].cat.codes
fire_data_1['day']=fire_data_1['day'].cat.codes
fire_data_1['size_category']=fire_data_1['size_category'].cat.codes
fire_data_1

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,size_category
0,7,0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,1
1,10,5,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,1
2,10,2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,1
3,7,0,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,1
4,7,3,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...
512,1,3,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0
513,1,3,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,0
514,1,3,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,0
515,1,2,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,1


In [ ]:
fire_data_1['month'].unique

<bound method Series.unique of 0       7
1      10
2      10
3       7
4       7
       ..
512     1
513     1
514     1
515     1
516     9
Name: month, Length: 517, dtype: int8>

## Data Preparation

In [ ]:
Std_scaler = StandardScaler()

In [ ]:
fire_data_2=Std_scaler.fit_transform(fire_data_1)
fire_data_2=pd.DataFrame(fire_data_2,columns=list(fire_data_1.columns.values))
fire_data_2

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,size_category
0,0.284222,-1.423121,-0.805959,-1.323326,-1.830477,-0.860946,-1.842640,0.411724,1.498614,-0.073268,-0.202020,0.606403
1,0.970871,1.176715,-0.008102,-1.179541,0.488891,-0.509688,-0.153278,-0.692456,-1.741756,-0.073268,-0.202020,0.606403
2,0.970871,-0.383187,-0.008102,-1.049822,0.560715,-0.509688,-0.739383,-0.692456,-1.518282,-0.073268,-0.202020,0.606403
3,0.284222,-1.423121,0.191362,-1.212361,-1.898266,-0.004756,-1.825402,3.233519,-0.009834,0.603155,-0.202020,0.606403
4,0.284222,0.136781,-0.243833,-0.931043,-1.798600,0.126966,-1.291012,3.356206,-1.238940,-0.073268,-0.202020,0.606403
...,...,...,...,...,...,...,...,...,...,...,...,...
512,-1.089076,0.136781,-1.640083,-0.846648,0.474768,-1.563460,1.536084,-0.753800,-0.736124,-0.073268,-0.100753,-1.649068
513,-1.089076,0.136781,-1.640083,-0.846648,0.474768,-1.563460,0.519019,1.638592,0.995798,-0.073268,0.651674,-1.649068
514,-1.089076,0.136781,-1.640083,-0.846648,0.474768,-1.563460,0.398350,1.577248,1.498614,-0.073268,-0.026532,-1.649068
515,-1.089076,-0.383187,0.680957,0.549003,0.269382,0.500176,1.156839,-0.140366,-0.009834,-0.073268,-0.202020,0.606403


In [ ]:
X = fire_data_2.drop(columns='size_category')
y=fire_data_1['size_category']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=5)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((361, 11), (156, 11), (361,), (156,))

## Model Preparation

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### **Step 1: Build the architecture**

In [ ]:
model=Sequential()
model.add(Dense(11,input_dim=11, kernel_initializer='uniform', activation='relu'))
model.add(Dense(units=10, kernel_initializer='uniform', activation='relu'))
model.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 11)                132       
                                                                 
 dense_19 (Dense)            (None, 10)                120       
                                                                 
 dense_20 (Dense)            (None, 1)                 11        
                                                                 
Total params: 263
Trainable params: 263
Non-trainable params: 0
_________________________________________________________________


## **Step 2 : Compile your architecture**

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

## **Step 3: Train your model**

In [ ]:
model.fit(X_train,y_train, epochs=100, batch_size=10)

## **6. Model Testing**

In [ ]:
model.evaluate(X_test,y_test)

5/5 [==============================] - 0s 3ms/step - loss: 0.0896 - accuracy: 0.9615


[0.08960368484258652, 0.9615384340286255]

## Tuning of Hyperparameters:- Activation Function and Kernel Initializer

In [ ]:
import tensorflow.keras.optimizers

In [ ]:
def create_model(activation_function,init,learning_rate):
  model=Sequential()
  model.add(Dense(11,kernel_initializer = init,activation=activation_function))
  model.add(Dense(12,kernel_initializer=init,activation=activation_function))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),loss='binary_crossentropy',metrics='accuracy')
  return model

model = KerasClassifier(build_fn=create_model,verbose=0,batch_size = 40,epochs = 10)
activation_function=['softmax','relu','tanh']
init=['uniform','normal','zero']
learning_rate = [0.001,0.01,0.1]


param_grids=dict(learning_rate = learning_rate,activation_function = activation_function,init = init)


grid= GridSearchCV(estimator=model,param_grid =param_grids,cv=KFold(),verbose = 10)
grid_result = grid.fit(X_train,y_train)

print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  # Remove the CWD from sys.path while we load stuff.


Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5; 1/27] START activation_function=softmax, init=uniform, learning_rate=0.001
[CV 1/5; 1/27] END activation_function=softmax, init=uniform, learning_rate=0.001;, score=0.781 total time=   1.4s
[CV 2/5; 1/27] START activation_function=softmax, init=uniform, learning_rate=0.001
[CV 2/5; 1/27] END activation_function=softmax, init=uniform, learning_rate=0.001;, score=0.861 total time=   1.0s
[CV 3/5; 1/27] START activation_function=softmax, init=uniform, learning_rate=0.001
[CV 3/5; 1/27] END activation_function=softmax, init=uniform, learning_rate=0.001;, score=0.611 total time=   0.9s
[CV 4/5; 1/27] START activation_function=softmax, init=uniform, learning_rate=0.001
[CV 4/5; 1/27] END activation_function=softmax, init=uniform, learning_rate=0.001;, score=0.639 total time=   0.9s
[CV 5/5; 1/27] START activation_function=softmax, init=uniform, learning_rate=0.001
[CV 5/5; 1/27] END activation_function=softmax, init=unifo

## Applying best parameters values to the final model

In [ ]:
model=Sequential()
model.add(Dense(11,input_dim=11, kernel_initializer='normal', activation='tanh'))
model.add(Dense(units=10, kernel_initializer='normal', activation='tanh'))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()


Model: "sequential_3037"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6040 (Dense)          (None, 11)                132       
                                                                 
 dense_6041 (Dense)          (None, 10)                120       
                                                                 
 dense_6042 (Dense)          (None, 1)                 11        
                                                                 
Total params: 263
Trainable params: 263
Non-trainable params: 0
_________________________________________________________________


## **Step 2 : Compile your architecture**

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),metrics=['accuracy'])

## **Step 3: Train your model**

In [ ]:
model.fit(X_train,y_train, epochs=50, batch_size=20)

Epoch 1/50
19/19 [==============================] - 0s 3ms/step - loss: 0.6131 - accuracy: 0.7175
Epoch 2/50
19/19 [==============================] - 0s 3ms/step - loss: 0.5251 - accuracy: 0.7562
Epoch 3/50
19/19 [==============================] - 0s 3ms/step - loss: 0.4263 - accuracy: 0.8033
Epoch 4/50
19/19 [==============================] - 0s 3ms/step - loss: 0.3193 - accuracy: 0.8670
Epoch 5/50
19/19 [==============================] - 0s 3ms/step - loss: 0.2950 - accuracy: 0.8837
Epoch 6/50
19/19 [==============================] - 0s 3ms/step - loss: 0.1733 - accuracy: 0.9169
Epoch 7/50
19/19 [==============================] - 0s 3ms/step - loss: 0.1124 - accuracy: 0.9695
Epoch 8/50
19/19 [==============================] - 0s 3ms/step - loss: 0.0970 - accuracy: 0.9640
Epoch 9/50
19/19 [==============================] - 0s 3ms/step - loss: 0.0792 - accuracy: 0.9751
Epoch 10/50
19/19 [==============================] - 0s 3ms/step - loss: 0.0743 - accuracy: 0.9778
Epoch 11/50
19/19 [

In [ ]:
model.evaluate(X_test,y_test)

5/5 [==============================] - 0s 3ms/step - loss: 0.0751 - accuracy: 0.9744


[0.07505737245082855, 0.9743589758872986]

## **Final model accuracy is 97.44 %**